In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import time
import datetime
import xarray
from mpl_toolkits.basemap import Basemap
import mpl_toolkits.basemap
import glob
import os
import shutil
import subprocess
import sys 
from scipy import signal
from netCDF4 import Dataset
%pylab inline
pylab.rcParams['figure.figsize'] = (16, 10) # Large figures
pylab.rcParams['image.cmap'] = 'seismic'
dpi=200;

In [ ]:
ds = xarray.open_dataset('/global/cscratch1/sd/lvroekel/sstRegion.nc')

In [ ]:
nin34 = ds.timeMonthly_avg_avgValueWithinOceanRegion_avgSurfaceTemperature[:,-2].values
nin34.shape

In [ ]:
times=ds.xtime_startMonthly.values

In [ ]:
#form annual cycle
nin34a=np.zeros_like(nin34)

avg = np.zeros(12)
for j in range(12):
    avg[j] = nin34[j::12].mean()
avgF = np.tile(avg,600/12)

temp = nin34[:] - avgF

#five month avg 
for j in range(2,598):
    nin34a[j] = np.mean(temp[j-2:j+2])

In [ ]:
#search for 6 months of > 0.4
LN = []
EN = []
ENL = []
LNL = []

lnlen = 0
enlen = 0
lns = 0
ens = 0
i=0
while i < 600:
    while nin34a[i] > 0.8 and i < 600:
        ens = i
        i += 1
        enlen+=1
    if enlen >= 6:
        EN.append(i-enlen)
        ENL.append(enlen)
        enlen = 0
    else:
        enlen = 0
    i+=1
i=0
while i < 600:
    while nin34a[i] < -0.8 and i < 600:
        i += 1
        lnlen+=1
        if i > 599:
            break
    if lnlen >= 6:
        LN.append(i-lnlen)
        LNL.append(lnlen)
        lnlen = 0
    else:
        lnlen = 0
    i+=1

In [ ]:
ensoEvents = np.zeros((len(EN),720,1440))
laninaEvents = np.zeros((len(LN),720,1440))

ensocount = 0
laninacount = 0
ds = xarray.open_dataset('/global/cscratch1/sd/lvroekel/concatfiles.nc')

enso = np.asarray(EN)
ensolen = np.asarray(ENL)

lanina = np.asarray(LN)
laninalen = np.asarray(LNL)

for jj in range(len(enso)):
    enstartM = np.mod(enso[jj],12)
    distNov = 11 - enstartM
    if distNov < ensolen[jj]:
        start = enso[jj]+distNov
        end = start + 2
    
    ensoEvents[ensocount,:,:] = np.nanmean(ds.timeMonthly_avg_activeTracers_temperature[start:end,0,:,:],axis=0)
    ensocount += 1

for jj in range(len(lanina)):
    enstartM = np.mod(lanina[jj],12)
    distNov = 11 - enstartM
    if distNov < laninalen[jj]:
        start = lanina[jj]+distNov
        end = start + 2
    
    laninaEvents[laninacount,:,:] = np.nanmean(ds.timeMonthly_avg_activeTracers_temperature[start:end,0,:,:],axis=0)
    laninacount += 1
    
#ensoEvents /= ensocount
#laninaEvents /= laninacount

In [ ]:
#get DJF mean
decSST = ds.timeMonthly_avg_activeTracers_temperature[11::12,0,:,:]
janSST = ds.timeMonthly_avg_activeTracers_temperature[0::12,0,:,:]
febSST = ds.timeMonthly_avg_activeTracers_temperature[1::12,0,:,:]
djfSST = (decSST + janSST + febSST) / 3.0

In [ ]:
#Plot high res data
ds = xarray.open_dataset('/global/cscratch1/sd/lvroekel/concatfiles.nc')

lat0=50

lon0=5

colormap = plt.get_cmap('seismic')
cmap = 'seismic'
clevs = [-2.5,-2.25,-2,-1.75,-1.5,-1.25,-1,-0.75,-.5,-.25,.25,.5,.75,1,1.25,1.5,1.75,2,2.25,2.5]
clevs2 =[-2.5,-2,-1.5,-1,-.5,.5,1,1.5,2,2.5]
#clevs = np.linspace(15,30,11)
#clevs2=clevs[::2]
cnorm = mpl.colors.BoundaryNorm(clevs, colormap.N)

pylab.rcParams['figure.figsize'] = (15, 10) # Large figures

plt.figure()
my_map = Basemap(projection='cyl',lon_0=180,resolution='c',fix_aspect=False)

dlat = 30
dlon=60

my_map.drawmapboundary(color='black')
my_map.drawparallels(np.arange(-90, 91, dlat), labels=[1, 0, 0, 0],fontsize=24);
my_map.drawmeridians(np.arange(-180, 180, dlon), labels=[0, 0, 1, 0],fontsize=24);

my_map.fillcontinents(color='lightgrey')
my_map.drawcoastlines()
## Uncomment one of these lines
#Make El Nino Composite
#tmp = ensoEvents[:,:,:].mean(axis=0) - djfSST.mean(axis=0)
#Make La Nina Composite
#tmp = laninaEvents[:,:,:].mean(axis=0) - djfSST.mean(axis=0)
#Make a high ENSO - low ENSO
#tmp = ensoEvents[:,:,:].mean(axis=0) -  laninaEvents[:,:,:].mean(axis=0)
lon22 = ds.lon.values
inds = np.where(lon22 < 0)[0]
lon22[inds]+=360
lonT = np.roll(lon22,720)
fld = np.roll(tmp,720,axis=1)

lon2,lat2 = np.meshgrid(lonT, ds.lat.values)
x, y = my_map(lon2,lat2)
dlat = 30
dlon=30

my_map.contour(x, y, fld, spacing='uniform', levels=clevs,colors='k')
CP = my_map.contourf(x, y, fld, cmap=cmap, spacing='uniform', levels=clevs,extend="both")

cbar = my_map.colorbar(CP, ticks=clevs2, boundaries=clevs)
cbar.ax.tick_params(labelsize=24, labelcolor='black')
cbar.set_label(r"$^o$C",fontsize=24)
plt.savefig('/global/homes/l/lvroekel/elninoComposite.png',bbox_inches='tight', pad_inches=0)